In [2]:
%matplotlib inline
from SALib.sample import saltelli
from SALib.analyze import sobol as sobol_analyze
from mesa.batchrunner import batch_run
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import pickle
from joblib import Parallel, delayed
from SALib.sample import sobol
from IPython.display import clear_output
import numpy as np
from itertools import product
import time

from model import SugarscapeG1mt

### Problem definition

In [3]:
problem = { 
    "num_vars": 10,
    "names": [
        "initial_population",
        "endowment_min",
        "endowment_max",
        "vision_min",
        "vision_max",
        "metabolism_min",
        "metabolism_max",
        "wealth_tax_system",
        "income_tax_system",
        "enable_staghunt"
    ],
    "bounds": [
        [50,   500],    # initial_population
        [5,    10],    # endowment_min
        [15,   30],    # endowment_max
        [1.0,     5.0],    # vision_min
        [5.0,     10.0],    # vision_max
        [0.1,  2.0],    # metabolism_min
        [2,     3.9],    # metabolism_max
        [0,     3],     # wealth_tax_system
        [0,     3],     # income_tax_system
        [0,     1]      # enable_staghunt
    ]
}
problem_staghunt = { 
    "num_vars": 12,
    "names": [
        "initial_population",
        "endowment_min",
        "endowment_max",
        "vision_min",
        "vision_max",
        "metabolism_min",
        "metabolism_max",
        "wealth_tax_system",
        "income_tax_system",
        "p_copy",
        "p_mutate",
        "enable_staghunt"
    ],
    "bounds": [
        [50,   500],    # initial_population
        [1,    10],    # endowment_min
        [15,   30],    # endowment_max
        [1.0,     5.0],    # vision_min
        [5.0,     10.0],    # vision_max
        [0.1,  2.0],    # metabolism_min
        [2,     3.9],    # metabolism_max
        [0,     3],     # wealth_tax_system
        [0,     3],     # income_tax_system
        [0.0,   1.0],   # p_copy
        [0.0,   0.20],  # p_mutate
        [0,     1]      # enable_staghunt
    ]
}

model_reporters = {
    "Gini": lambda m:m.datacollector.get_model_vars_dataframe()['Gini'].iloc[-1],
    "Avg Wealth": lambda m:m.datacollector.get_model_vars_dataframe()['Average Wealth'].iloc[-1],

}

wealth_tax_map = {
    0: "none",  # No wealth tax
    1: "proportional",  # Proportional
    2: "progressive",  # Progressive tax
    3: "degressive"   # Regressive tax
}

income_tax_map = {
    0: "none",  # No income tax
    1: "proportional",  # Proportional income tax
    2: "progressive",  # Progressive income tax
    3: "degressive"   # Regressive income tax
}

integer_vars = {
    "initial_population",
    "endowment_min",
    "endowment_max",
    "vision_min",
    "vision_max",
    "metabolism_min",
    "metabolism_max",
    "wealth_tax_period",
    "wealth_tax_system",
    "income_tax_system",
    "enable_staghunt"
}


integer_vars_staghunt = {
    "initial_population",
    "endowment_min",
    "endowment_max",
    "vision_min",
    "vision_max",
    "metabolism_min",
    "metabolism_max",
    "wealth_tax_period",
    "wealth_tax_system",
    "income_tax_system",
    "p_copy",
    "p_mutate",
    "enable_staghunt"
}


discrete_factors = {
    "wealth_tax_system": [0, 1, 2, 3],
    "income_tax_system": [0, 1, 2, 3],
    "enable_staghunt":   [0, 1],
}
# Build reduced Sobol problem for the other 12 vars
cont_vars   = [n for n in problem["names"] if n not in discrete_factors]
cont_bounds = [
    b for (n, b) in zip(problem["names"], problem["bounds"])
         if n not in discrete_factors
]
reduced_problem = {
    "num_vars": len(cont_vars),
    "names":    cont_vars,
    "bounds":   cont_bounds
}


cont_vars_staghunt = [n for n in problem_staghunt["names"] if n not in discrete_factors]
cont_bounds_staghunt = [
    b for (n, b) in zip(problem_staghunt["names"], problem_staghunt["bounds"])
         if n not in discrete_factors
]
reduced_problem_staghunt = {
    "num_vars": len(cont_vars_staghunt),
    "names":    cont_vars_staghunt,
    "bounds":   cont_bounds_staghunt
}

In [6]:
measure = "Gini" # "Gini" # or "Avg Wealth"
second_order = True  # or: "True"Oh

# --- 3) Sobol GSA ---
replicates = 2
max_steps = 10
distinct_SA = 2

In [7]:

param_values = sobol.sample(problem, distinct_SA, calc_second_order=second_order)
total_runs = replicates * len(param_values)
print(f"Total runs for Sobol GSA NO staghunt: {total_runs}")


param_values_staghunt = sobol.sample(problem_staghunt, distinct_SA, calc_second_order=second_order)
total_runs = replicates * len(param_values_staghunt)
print(f"Total runs for Sobol GSA  staghunt: {total_runs}")

Total runs for Sobol GSA NO staghunt: 88
Total runs for Sobol GSA  staghunt: 104


In [8]:
def run_mixed(run_id, params):
    # cast ints/floats just as before
    for name in integer_vars:
        if name in params:
            params[name] = int(round(params[name]))
    #params["flat_rate"]           = float(params["flat_rate"])
    #params["income_tax_flat_rate"]= float(params["income_tax_flat_rate"])
    #params["p_copy"]              = float(params["p_copy"])
    #params["p_mutate"]            = float(params["p_mutate"])
    # ensure min≤max
    for low, high in [("endowment_min","endowment_max"),
                        ("vision_min",   "vision_max"),
                        ("metabolism_min","metabolism_max")]:
        lo, hi = sorted((params[low], params[high]))
        params[low], params[high] = int(lo), int(hi)
    # map tax‐system integers to names
    params["wealth_tax_system"] = wealth_tax_map[params["wealth_tax_system"]]
    params["income_tax_system"]= income_tax_map[params["income_tax_system"]]
    # run your model
    print(params)
    out = batch_run(
        SugarscapeG1mt,
        parameters=params,
        iterations=1,
        max_steps=max_steps,
        data_collection_period=-1,
        display_progress=False
    )[0]
    return {
        **params,
        "RunId":    run_id,
        "Gini":     out["Gini"],
        "Avg Wealth": out["Average Wealth"]
    }

In [9]:
all_records = []
for w_sys in  discrete_factors["wealth_tax_system"]: 
    print("running for ", w_sys)
    # 2a) sample continuous part
    cont_samples = sobol.sample(
        reduced_problem,
        N=distinct_SA,
        calc_second_order=second_order
    )

    # 2b) build tasks
    tasks = []
    for run_id, cont_vals in enumerate(cont_samples):
        # merge into a single params dict
        full_params = {n: v for n, v in zip(cont_vars, cont_vals)}

        full_params.update({
            "wealth_tax_system": w_sys,
            "income_tax_system": w_sys,
            "enable_staghunt":   False
        })
        tasks.append((run_id, full_params))
    slice_results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_mixed)(rid, p) for rid, p in tasks
    )

    record = {
        "discrete_setting": (w_sys, w_sys, False),
        "samples":          cont_samples,
        "results":          slice_results
    }
    all_records.append(record)

    # Save this slice to a separate file
    output_path = f"sobol_gsa_setting_w{w_sys}_hunt0.pkl"
    with open(output_path, "wb") as f:
        pickle.dump(record, f)
    print(f"Saved setting (w_sys={w_sys}, hunt_flag=0) to '{output_path}'.")

print(f"\nAll {len(all_records)} settings completed and saved separately.")

running for  0


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  13 out of  32 | elapsed:    6.0s remaining:    8.8s
[Parallel(n_jobs=-1)]: Done  17 out of  32 | elapsed:    6.9s remaining:    6.1s
[Parallel(n_jobs=-1)]: Done  21 out of  32 | elapsed:    7.6s remaining:    3.9s
[Parallel(n_jobs=-1)]: Done  25 out of  32 | elapsed:    8.0s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  29 out of  32 | elapsed:    8.3s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:    8.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Saved setting (w_sys=0, hunt_flag=0) to 'sobol_gsa_setting_w0_hunt0.pkl'.
running for  1


ValueError: min() iterable argument is empty

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
import pickle
import os

# ---- Import your model ----
# from your_model_module import SugarscapeModel, model_reporters

# Create output directory
os.makedirs("gsa_outputs", exist_ok=True)

########################################
# General Settings
########################################

N_REPLICATIONS = 2
N_SAMPLES = 2 


model_reporters = {
    "Gini": lambda m: m.datacollector.get_model_vars_dataframe()['Gini'].iloc[-1],
    "Avg Wealth": lambda m: m.datacollector.get_model_vars_dataframe()['Average Wealth'].iloc[-1],
}

tax_systems = {
    0: "none",
    1: "proportional",
    2: "progressive",
    3: "degressive"
}


########################################
# Model Execution Function
########################################

def run_model(params_dict, n_replications=1):
    results = {k: [] for k in model_reporters.keys()}
    for _ in range(n_replications):
        model = SugarscapeG1mt(**params_dict, max_steps=20)
        model.run_model()
        for key, func in model_reporters.items():
            results[key].append(func(model))
    return {k: np.mean(v) for k, v in results.items()}


########################################
# Sobol Runner Function
########################################

def run_sobol(problem_def, fixed_params, integer_vars_set, n_samples=N_SAMPLES, label=""):
    param_names = problem_def["names"]
    bounds = problem_def["bounds"]

    param_values = saltelli.sample(problem_def, n_samples, calc_second_order=False)

    outputs = {k: [] for k in model_reporters.keys()}

    for row in tqdm(param_values, desc=f"Running {label}"):
        params = dict(zip(param_names, row))
        params.update(fixed_params)

        for var in integer_vars_set:
            if var in params:
                params[var] = int(round(params[var]))

        out = run_model(params, n_replications=N_REPLICATIONS)
        for k in outputs.keys():
            outputs[k].append(out[k])

    for k in outputs.keys():
        outputs[k] = np.array(outputs[k])

    sobol_results = {}
    for k in outputs.keys():
        sobol_results[k] = sobol.analyze(
            problem_def, outputs[k], calc_second_order=False, print_to_console=False
        )
    
    return sobol_results


########################################
# Worker Function for Parallelization
########################################

def worker(task):
    label, problem_def, fixed_params, integer_vars_set, is_staghunt = task
    print(f"\n--- Starting GSA for {label} ---")

    results = run_sobol(
        problem_def=problem_def,
        fixed_params=fixed_params,
        integer_vars_set=integer_vars_set,
        n_samples=N_SAMPLES,
        label=label
    )

    output_path = f"gsa_outputs/gsa_{label}.pkl"
    with open(output_path, "wb") as f:
        pickle.dump(results, f)

    print(f"--- Finished GSA for {label}. Saved to {output_path} ---")
    return label


########################################
# Build Tasks List
########################################

tasks = []

# Tax systems (staghunt disabled)
for tax_code, tax_name in tax_systems.items():
    fixed_params = {
        "wealth_tax_system": tax_code,
        "income_tax_system": tax_code,
        "enable_staghunt": 0
    }
    tasks.append((
        f"tax_{tax_name}",
        reduced_problem,
        fixed_params,
        integer_vars,
        False
    ))

# Staghunt enabled (no tax)
fixed_params_staghunt = {
    "wealth_tax_system": 0,
    "income_tax_system": 0,
    "enable_staghunt": 1
}
tasks.append((
    "staghunt_enabled",
    reduced_problem_staghunt,
    fixed_params_staghunt,
    integer_vars_staghunt,
    True
))


########################################
# Run in Parallel
########################################

if __name__ == "__main__":
    n_cpus = max(mp.cpu_count() - 1, 1)
    print(f"Running on {n_cpus} CPUs")

    with mp.Pool(processes=n_cpus) as pool:
        results = pool.map(worker, tasks)

    print("\nAll GSA jobs completed.")


Running on 11 CPUs
